In [1]:
import uproot
import hist

In [2]:
from core import *

In [3]:
# first, lets define a few configurations

# these are usually 17 mass points, 18 with 3000 included!
masspoints = [700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2250, 2500, 2750, 3000]
signalscalings = [1./100.] * 18 # scaling all down by factor 100, but ONLY DO THIS FOR SR!!!
generalscaling = 1 # another scaling that is simply applied to *everything*, except signal. don't change this :D
region = "VR" # options are SR and VR
spinpostfix = "_Spin32" # options are "" and "_Spin32"
years = ["UL16preVFP", "UL16postVFP", "UL17", "UL18"]
years_to_create = ["UL16preVFP", "UL16postVFP", "UL17", "UL18"]
channels = ["ele", "mu"]   # splitting electron and muon channel

doSystematics = True
doJECJER = True
doJECsmooth = True
doPDF_MCscale = True
doDatadriven = True
varyDatadrivenBtagAlone = True # only implementing true at the moment
datadriven_systs = {
    "Func" : "datadrivenFitFunction",
    "fitstat" : "datadrivenFitStat"
    #"channel" : "datadrivenChannelSplit",
    #"year" : "datadrivenYearSplit",
}
autoMCstatsParam = 0

if not doSystematics:
    datadriven_systs = {}

assert( len(masspoints) == len(signalscalings) )

signalscalings = [this_scaling / generalscaling for this_scaling in signalscalings]

In [4]:
# the main löööp

for masspoint, scaling in zip(masspoints, signalscalings):

    print("Starting masspoint " + str(masspoint) +".")

    MC_samples = ["TstarTstar" + spinpostfix + "_M-" + str(masspoint), "TTbar", "ST"] # only top backgrounds are taken from MC
    if not doDatadriven:
        MC_samples.append("DYJets")
        MC_samples.append("QCD")
        MC_samples.append("VV")
        MC_samples.append("WJets")
    
    # define normalisation uncertainties for samples and years
    if doSystematics:
        norm_uncertainties = {
            "lumi_13TeV_UL16": {
              p: {"UL16preVFP": 1.01, "UL16postVFP": 1.01, "UL17": '-', "UL18": '-'} for p in MC_samples
            },
            "lumi_13TeV_UL17": {
              p: {"UL16preVFP": '-', "UL16postVFP": '-', "UL17": 1.02, "UL18": '-'} for p in MC_samples
            },
            "lumi_13TeV_UL18": {
              p: {"UL16preVFP": '-', "UL16postVFP": '-', "UL17": '-', "UL18": 1.015} for p in MC_samples
            },
            "lumi_13TeV_UL16UL17UL18": {
              p: {"UL16preVFP": 1.006, "UL16postVFP": 1.006, "UL17": 1.009, "UL18": 1.02} for p in MC_samples
            },
            "lumi_13TeV_UL17UL18": {
              p: {"UL16preVFP": '-', "UL16postVFP": '-', "UL17": 1.006, "UL18": 1.002} for p in MC_samples
            },
        }

        # shape uncertainties that can be read as "pt_ST_<systematic><Variation>" where variation is "Up" or "Down"
        # these, by construction, are only relevant for the systematics taken from MC
        # planned structure: dict of variations, containing a tuple: first element defines samples, second defined corr.
        shape_uncertanties = {
            "pu": [ MC_samples , [years] ], # fully correlated
            "prefiring": [ MC_samples ,  [["UL16preVFP"], ["UL16postVFP"], ["UL17"], ["UL18"]] ], # uncorrelated
            "btagging_hf": [ MC_samples , [years] ], # fully correlated
            "btagging_hfstats1": [ MC_samples , [["UL16preVFP"], ["UL16postVFP"], ["UL17"], ["UL18"]] ], # uncorrelated
            "btagging_hfstats2": [ MC_samples , [["UL16preVFP"], ["UL16postVFP"], ["UL17"], ["UL18"]] ], # uncorrelated
            "btagging_lf": [ MC_samples , [years] ], # fully correlated
            "btagging_lfstats1": [ MC_samples , [["UL16preVFP"], ["UL16postVFP"], ["UL17"], ["UL18"]] ], # uncorrelated
            "btagging_lfstats2": [ MC_samples , [["UL16preVFP"], ["UL16postVFP"], ["UL17"], ["UL18"]] ], # uncorrelated
            "btagging_cferr1": [ MC_samples , [years] ], # fully correlated
            "btagging_cferr2": [ MC_samples , [years] ], # fully correlated
            "btagYield": [ MC_samples , [years] ], # fully correlated
            "sfelec_id": [ MC_samples , [years] ], # fully correlated
            "sfelec_reco": [ MC_samples , [years] ], # fully correlated
            "sfelec_trigger": [ MC_samples , [["UL16preVFP"], ["UL16postVFP"], ["UL17"], ["UL18"]] ], # uncorrelated
            "sfmu_id": [ MC_samples , [["UL16preVFP"], ["UL16postVFP"], ["UL17"], ["UL18"]] ], # uncorrelated
            "sfmu_iso": [ MC_samples , [["UL16preVFP"], ["UL16postVFP"], ["UL17"], ["UL18"]] ], # uncorrelated
            "sfmu_trigger": [ MC_samples , [["UL16preVFP"], ["UL16postVFP"], ["UL17"], ["UL18"]] ], # uncorrelated
            "topPt": [ MC_samples , [years] ], # fully correlated
            "decorrelation": [ MC_samples[1:] , [years] ], # fully correlated
            # attention last line assumes signal is first!!!
        }
    else:
        norm_uncertainties = {}
        shape_uncertanties = {}
    
    for year in years_to_create:
        
        print("Working on " + year + "...")
        
        for channel in channels:
            
            print("in channel " + channel + ".")
            
            datacard = Datacard(year,
                                masspoint,
                                "pt_ST",
                                channel,
                                region,
                                MC_samples,
                                norm_uncertainties,
                                shape_uncertanties,
                                spinpostfix,
                                scaling = scaling,
                                generalscaling = generalscaling,
                                doJECJER = doJECJER,
                                doJECsmooth = doJECsmooth,
                                doPDF_MCscale = doPDF_MCscale,
                                doDatadriven = doDatadriven,
                                varyDatadrivenBtagAlone = varyDatadrivenBtagAlone,
                                datadriven_systs = datadriven_systs,
                                autoMCstatsParam = autoMCstatsParam,
                               )
    
            datacard.create_rootfile(shape_uncertanties)
        
print("Done!")

Starting masspoint 700.
Working on UL16preVFP...
in channel ele.
in channel mu.
Working on UL16postVFP...
in channel ele.
in channel mu.
Working on UL17...
in channel ele.
in channel mu.
Working on UL18...
in channel ele.
in channel mu.
Starting masspoint 800.
Working on UL16preVFP...
in channel ele.
in channel mu.
Working on UL16postVFP...
in channel ele.
in channel mu.
Working on UL17...
in channel ele.
in channel mu.
Working on UL18...
in channel ele.
in channel mu.
Starting masspoint 900.
Working on UL16preVFP...
in channel ele.
in channel mu.
Working on UL16postVFP...
in channel ele.
in channel mu.
Working on UL17...
in channel ele.
in channel mu.
Working on UL18...
in channel ele.
in channel mu.
Starting masspoint 1000.
Working on UL16preVFP...
in channel ele.
in channel mu.
Working on UL16postVFP...
in channel ele.
in channel mu.
Working on UL17...
in channel ele.
in channel mu.
Working on UL18...
in channel ele.
in channel mu.
Starting masspoint 1100.
Working on UL16preVFP...
i